In [26]:
import os
import sys
sys.path.append('..')
import json
from PIL import Image
import torch
from torch.utils.data import Dataset
from core.model import PretrainedModel
import clip
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
import numpy as np
from sklearn.decomposition import PCA

%load_ext autoreload
%autoreload 2

In [27]:
data_dir = os.path.join('data/')

In [28]:
os.path.join(data_dir, 'train_clip_temp.jsonl')

'data/train_clip_temp.jsonl'

# Setup for model using clip (WIP)

### 0. Get the data

Download the dataset from https://hatefulmemeschallenge.com/ 

### 1. Create the dataset
- Check that the data is in base_dir/data/ otherwise change data_dir in previous cell
- Use clip to create the jsonl with the embeddings needed for train, evaluation and test
- (opt.) Create the jsonl with the embeddings of additional classes 
- (opt.) Run kmeans to get n clusters in the embedding space

In [25]:
from scripts.compute_embeddings_clip import makeClipDataset
# Use clip to create the jsonl with the embeddings needed for train, evaluation and test

makeClipDataset(os.path.join(data_dir, 'train_clip.jsonl'), os.path.join(data_dir, 'train.jsonl'))
makeClipDataset(os.path.join(data_dir, 'eval_clip.jsonl'), os.path.join(data_dir, 'dev.jsonl'))
makeClipDataset(os.path.join(data_dir, 'test_clip.jsonl'), os.path.join(data_dir, 'test_seen.jsonl'))



file already exists, remove it to proceed: data/train_clip_temp.jsonl
file already exists, remove it to proceed: data/eval_clip_temp.jsonl
file not found: data/test_seen.jsonl


In [ ]:
from scripts.compute_embeddings_clip import makeClipClassesEmbeddings
# (opt.) Create the jsonl with the embeddings of additional classes 

classes = ['Hate Speech', 'Insult', 'Sarcastic', 'Blasphemy', 'Swearing', 'Threat', 'Sexist', 'Racist', 'Offensive', 'Abusive', 'Profane', 'Homophobic', 'Xenophobic', 'Misogynistic', 'Violent', 'Harassment', 'Cyberbullying', 'Trolling', 'Stalking', 'Doxxing', 'Grooming', 'Neutral', 'Positive', 'Negative']
#classes += ['Ass', 'Asshole', 'Homosexual', 'Homophobic', 'Racist', 'Gay', 'Lgbt', 'Jew', 'Jewish', 'Anti-semitic', 'Chink', 'Muslims', 'Muslim', 'Isis', 'Islamophobe', 'homophobe ', 'Bombing', 'Sexyhot', 'Bastard', 'Bitch', 'Fucker', 'Cunt', 'Damn', 'Fuck', 'Goddamn', 'Shit', 'Motherfucker', 'Nigga', 'Nigger', 'Prick', 'Shit', 'shit ass', 'Shitass', 'son of a bitch', 'Whore', 'Thot', 'Slut', 'Faggot', 'Dick', 'Pussy', 'Penis', 'Vagina', 'Negro', 'Coon', 'Bitched', 'Sexist', 'Freaking', 'Cock', 'Sucker', 'Lick', 'Licker', 'Rape', 'Molest', 'Anal', 'Buttrape', 'Coont', 'Cancer', 'Sex', 'Retard', 'Fuckface', 'Dumbass', '5h1t', '5hit', 'A_s_s', 'a2m', 'a55', 'adult', 'amateur', 'anal', 'anal impaler', 'anal leakage', 'anilingus', 'anus', 'ar5e', 'arrse', 'arse', 'arsehole', 'ass', 'ass fuck', 'asses', 'assfucker', 'ass-fucker', 'assfukka', 'asshole', 'asshole', 'assholes', 'assmucus', 'assmunch', 'asswhole', 'autoerotic', 'b!tch', 'b00bs', 'b17ch', 'b1tch', 'ballbag', 'ballsack', 'bang (one\'s) box', 'bangbros', 'bareback', 'bastard', 'beastial', 'beastiality', 'beef curtain', 'bellend', 'bestial', 'bestiality', 'bi+ch', 'biatch', 'bimbos', 'birdlock', 'bitch', 'bitch tit', 'bitcher', 'bitchers', 'bitches', 'bitchin', 'bitching', 'bloody', 'blow job', 'blow me', 'blow mud', 'blowjob', 'blowjobs', 'blue waffle', 'blumpkin', 'boiolas', 'bollock', 'bollok', 'boner', 'boob', 'boobs', 'booobs', 'boooobs', 'booooobs', 'booooooobs', 'breasts', 'buceta', 'bugger', 'bum', 'bunny fucker', 'bust a load', 'busty', 'butt', 'butt fuck', 'butthole', 'buttmuch', 'buttplug', 'c0ck', 'c0cksucker', 'carpet muncher', 'carpetmuncher', 'cawk', 'chink', 'choade', 'chota bags', 'cipa', 'cl1t', 'clit', 'clit licker', 'clitoris', 'clits', 'clitty litter', 'clusterfuck', 'cnut', 'cock', 'cock pocket', 'cock snot', 'cockface', 'cockhead', 'cockmunch', 'cockmuncher', 'cocks', 'cocksuck ', 'cocksucked ', 'cocksucker', 'cock-sucker', 'cocksucking', 'cocksucks ', 'cocksuka', 'cocksukka', 'cok', 'cokmuncher', 'coksucka', 'coon', 'cop some wood', 'cornhole', 'corp whore', 'cox', 'cum', 'cum chugger', 'cum dumpster', 'cum freak', 'cum guzzler', 'cumdump', 'cummer', 'cumming', 'cums', 'cumshot', 'cunilingus', 'cunillingus', 'cunnilingus', 'cunt', 'cunt hair', 'cuntbag', 'cuntlick ', 'cuntlicker ', 'cuntlicking ', 'cunts', 'cuntsicle', 'cunt-struck', 'cut rope', 'cyalis', 'cyberfuc', 'cyberfuck ', 'cyberfucked ', 'cyberfucker', 'cyberfuckers', 'cyberfucking ', 'd1ck', 'damn', 'dick', 'dick hole', 'dick shy', 'dickhead', 'dildo', 'dildos', 'dink', 'dinks', 'dirsa', 'dirty Sanchez', 'dlck', 'dog-fucker', 'doggie style', 'doggiestyle', 'doggin', 'dogging', 'donkeyribber', 'doosh', 'duche', 'dyke', 'eat a dick', 'eat hair pie', 'ejaculate', 'ejaculated', 'ejaculates ', 'ejaculating ', 'ejaculatings', 'ejaculation', 'ejakulate', 'erotic', 'f u c k', 'f u c k e r', 'f_u_c_k', 'f4nny', 'facial', 'fag', 'fagging', 'faggitt', 'faggot', 'faggs', 'fagot', 'fagots', 'fags', 'fanny', 'fannyflaps', 'fannyfucker', 'fanyy', 'fatass', 'fcuk', 'fcuker', 'fcuking', 'feck', 'fecker', 'felching', 'fellate', 'fellatio', 'fingerfuck ', 'fingerfucked ', 'fingerfucker ', 'fingerfuckers', 'fingerfucking ', 'fingerfucks ', 'fist fuck', 'fistfuck', 'fistfucked ', 'fistfucker ', 'fistfuckers ', 'fistfucking ', 'fistfuckings ', 'fistfucks ', 'flange', 'flog the log', 'fook', 'fooker', 'fuck hole', 'fuck puppet', 'fuck trophy', 'fuck yo mama', 'fuck', 'fucka', 'fuck-ass', 'fuck-bitch', 'fucked', 'fucker', 'fuckers', 'fuckhead', 'fuckheads', 'fuckin', 'fucking', 'fuckings', 'fuckingshitmotherfucker', 'fuckme ', 'fuckmeat', 'fucks', 'fucktoy', 'fuckwhit', 'fuckwit', 'fudge packer', 'fudgepacker', 'fuk', 'fuker', 'fukker', 'fukkin', 'fuks', 'fukwhit', 'fukwit', 'fux', 'fux0r', 'gangbang', 'gangbang', 'gang-bang', 'gangbanged ', 'gangbangs ', 'gassy ass', 'gaylord', 'gaysex', 'goatse', 'god', 'god damn', 'god-dam', 'goddamn', 'goddamned', 'god-damned', 'ham flap', 'hardcoresex ', 'hell', 'heshe', 'hoar', 'hoare', 'hoer', 'homo', 'homoerotic', 'hore', 'horniest', 'horny', 'hotsex', 'how to kill', 'how to murdep', 'jackoff', 'jack-off ', 'jap', 'jerk', 'jerk-off ', 'jism', 'jiz ', 'jizm ', 'jizz', 'kawk', 'kinky Jesus', 'knob', 'knob end', 'knobead', 'knobed', 'knobend', 'knobend', 'knobhead', 'knobjocky', 'knobjokey', 'kock', 'kondum', 'kondums', 'kum', 'kummer', 'kumming', 'kums', 'kunilingus', 'kwif', 'l3i+ch', 'l3itch', 'labia', 'LEN', 'lmao', 'lmfao', 'lmfao', 'lust', 'lusting', 'm0f0', 'm0fo', 'm45terbate', 'ma5terb8', 'ma5terbate', 'mafugly', 'masochist', 'masterb8', 'masterbat*', 'masterbat3', 'masterbate', 'master-bate', 'masterbation', 'masterbations', 'masturbate', 'mof0', 'mofo', 'mo-fo', 'mothafuck', 'mothafucka', 'mothafuckas', 'mothafuckaz', 'mothafucked ', 'mothafucker', 'mothafuckers', 'mothafuckin', 'mothafucking ', 'mothafuckings', 'mothafucks', 'mother fucker', 'mother fucker', 'motherfuck', 'motherfucked', 'motherfucker', 'motherfuckers', 'motherfuckin', 'motherfucking', 'motherfuckings', 'motherfuckka', 'motherfucks', 'muff', 'muff puff', 'mutha', 'muthafecker', 'muthafuckker', 'muther', 'mutherfucker', 'n1gga', 'n1gger', 'nazi', 'need the dick', 'nigg3r', 'nigg4h', 'nigga', 'niggah', 'niggas', 'niggaz', 'nigger', 'niggers ', 'nob', 'nob jokey', 'nobhead', 'nobjocky', 'nobjokey', 'numbnuts', 'nut butter', 'nutsack', 'omg', 'orgasim ', 'orgasims ', 'orgasm', 'orgasms ', 'p0rn', 'pawn', 'pecker', 'penis', 'penisfucker', 'phonesex', 'phuck', 'phuk', 'phuked', 'phuking', 'phukked', 'phukking', 'phuks', 'phuq', 'pigfucker', 'pimpis', 'piss', 'pissed', 'pisser', 'pissers', 'pisses ', 'pissflaps', 'pissin ', 'pissing', 'pissoff ', 'poop', 'porn', 'porno', 'pornography', 'pornos', 'prick', 'pricks ', 'pron', 'pube', 'pusse', 'pussi', 'pussies', 'pussy', 'pussy fart', 'pussy palace', 'pussys ', 'queaf', 'queer', 'rectum', 'retard', 'rimjaw', 'rimming', 's hit', 's.o.b.', 's_h_i_t', 'sadism', 'sadist', 'sandbar', 'sausage queen', 'schlong', 'screwing', 'scroat', 'scrote', 'scrotum', 'semen', 'sex', 'sh!+', 'sh!t', 'sh1t', 'shag', 'shagger', 'shaggin', 'shagging', 'shemale', 'shi+', 'shit', 'shit fucker', 'shitdick', 'shite', 'shited', 'shitey', 'shitfuck', 'shitfull', 'shithead', 'shiting', 'shitings', 'shits', 'shitted', 'shitter', 'shitters ', 'shitting', 'shittings', 'shitty ', 'skank', 'slope', 'slut', 'slut bucket', 'sluts', 'smegma', 'smut', 'snatch', 'son-of-a-bitch', 'spac', 'spunk', 't1tt1e5', 't1tties', 'teets', 'teez', 'testical', 'testicle', 'tit', 'tit wank', 'titfuck', 'tits', 'titt', 'tittie5', 'tittiefucker', 'titties', 'tittyfuck', 'tittywank', 'titwank', 'tosser', 'turd', 'tw4t', 'twat', 'twathead', 'twatty', 'twunt', 'twunter', 'v14gra', 'v1gra', 'vagina', 'viagra', 'vulva', 'w00se', 'wang', 'wank', 'wanker', 'wanky', 'whoar', 'whore', 'willies', 'willy', 'wtf', 'xrated', 'xxx', 'sucker', 'dumbass', 'Kys', 'Kill', 'Die', 'Cliff', 'Bridge', 'Shooting', 'Shoot', 'Bomb', 'Terrorist', 'Terrorism', 'Bombed', 'Trump', 'Maga', 'Conservative', 'Make america great again', 'Far right', 'Necrophilia', 'Mongoloid', 'Furfag', 'Cp', 'Pedo', 'Pedophile', 'Pedophilia', 'Child predator', 'Predatory', 'Depression', 'Cut myself', 'I want to die', 'Fuck life', 'Redtube', 'Loli', 'Lolicon', 'Cub']

classes_embeddings = makeClipClassesEmbeddings(os.path.join(data_dir, 'classes_clip.jsonl'), classes)


In [ ]:
# (opt.) Run kmeans to get n clusters in the embedding space
X = np.array([el['embedding'][0].cpu().detach().numpy().tolist() for el in classes_embeddings])
print(X.shape)
pca = PCA(n_components=2).fit(X)
reduced_data = pca.transform(X)

kmeans = KMeans(n_clusters=64, random_state=0, n_init="auto").fit(reduced_data)
centers = pca.inverse_transform(kmeans.cluster_centers_)

with open(os.path.join(data_dir,'classes_clip_centers.jsonl'), 'w') as f:
  for c in tqdm(centers, total=len(centers)):
    text = c
    f.write(json.dumps({'embedding': text.tolist()}) + '\n')

### 2. Test the model 

Run the following cell making sure that DATA_DIR in training_clip_freeze is equal to the one where you saved the files

Current implementation is based on the paper https://aclanthology.org/2022.nlp4pi-1.20.pdf implemented at https://github.com/gokulkarthik/hateclipper

However the results are not the one described in it even though the architecture should be similar. Obtained AUC-ROC is around 0.70% while the one described in the paper is around 84%



In [21]:
!cd ./..
!python training_clip_freeze.py

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 1, 1028]         790,532
            Linear-2              [-1, 1, 1028]         790,532
           Dropout-3              [-1, 1, 1028]               0
           Dropout-4              [-1, 1, 1028]               0
            Linear-5              [-1, 1, 1028]       1,057,812
              ReLU-6              [-1, 1, 1028]               0
              ReLU-7              [-1, 1, 1028]               0
              ReLU-8              [-1, 1, 1028]               0
              ReLU-9              [-1, 1, 1028]               0
          Dropout-10              [-1, 1, 1028]               0
          Dropout-11              [-1, 1, 1028]               0
          Dropout-12              [-1, 1, 1028]               0
          Dropout-13              [-1, 1, 1028]               0
           Linear-14                 [-